In [15]:
import pandas as pd
import numpy as np  # 调用numpy
import xarray as xr
from eofs.standard import Eof
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

In [16]:
def createmap():  ###############################################生成地图##########################################################
    box = [120, 300, -20, 20]  # 经度维度
    scale = '110m'  # 地图分辨率
    xstep = 20  # 下面标注经纬度的步长
    ystep = 10
    proj = ccrs.PlateCarree(central_longitude=240)  # 确定地图投影
    fig = plt.figure(figsize=(9, 6))  ###生成底图
    ax = fig.subplots(1, 1, subplot_kw={'projection': proj})  # 确定子图，与grads的类似
    ##海岸线
    ax.coastlines(scale)
    ax.set_extent(box, ccrs.PlateCarree())
    # 标注坐标轴
    ax.set_xticks(np.arange(box[0], box[1] + xstep, xstep), crs=ccrs.PlateCarree())
    ax.set_yticks(np.arange(box[2], box[3] + ystep, ystep), crs=ccrs.PlateCarree())
    # 经纬度格式，把0经度设置不加E和W
    lon_formatter = LongitudeFormatter(zero_direction_label=False)
    lat_formatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)
    ############################################################################################################
    return ax, fig

In [17]:
plt.rcParams['font.sans-serif'] = ['SimHei']  ###防止无法显示中文并设置黑体
plt.rcParams['axes.unicode_minus'] = False  ###用来正常显示负号

In [18]:
iosst = xr.open_dataset('D:\\grads\\TongJi\\NCEP_IOSST_30y_Wt.nc')
tpsst = xr.open_dataset('D:\\grads\\TongJi\\NCEP_TPSST_30y_Wt.nc')

In [19]:
tp = tpsst['st'][:, :, :]
lat = tpsst['lat'][:]
lon = tpsst['lon'][:]
t = tpsst['time'][:]
tp = tp.where(tp < 9999, np.NAN)
avetp = tp.mean(dim="time", skipna=True)

In [20]:
jupingtp = np.zeros((len(t), len(lat), len(lon)))
for t_i in range(len(t)):
    jupingtp[t_i, :, :] = tp[t_i, :, :] - avetp
a = Eof(jupingtp)

In [21]:
##取前几个模态
n = 10
eof = a.eofsAsCorrelation(neofs=n)
pc = a.pcs(npcs=n, pcscaling=1)
var = a.varianceFraction(neigs=n)

In [22]:
##
ax, fig = createmap()
colorbar = ax.contourf(lon, lat, eof[0, :, :], cmap='RdBu', transform=ccrs.PlateCarree())
lines = ax.contour(lon, lat, eof[0, :, :], colors='k', linestyles='-', transform=ccrs.PlateCarree())
plt.clabel(lines, inline=True, fontsize=8, fmt='%.2f')
plt.colorbar(colorbar, extendrect='True', pad=0.1, fraction=0.04, shrink=1, location="bottom")
ax.set_title('第一模态')
plt.tight_layout()
plt.savefig("D:\\grads\\TongJi\\ex6\\eof1.1.png")
plt.close()
ax, fig = createmap()
colorbar = ax.contourf(lon, lat, eof[1, :, :], cmap='RdBu', transform=ccrs.PlateCarree())
lines = ax.contour(lon, lat, eof[1, :, :], colors='k', linestyles='-', transform=ccrs.PlateCarree())
plt.clabel(lines, inline=True, fontsize=8, fmt='%.2f')
plt.colorbar(colorbar, extendrect='True', pad=0.1, fraction=0.04, shrink=1, location="bottom")
ax.set_title('第二模态')
plt.tight_layout()
plt.savefig("D:\\grads\\TongJi\\ex6\\eof2.1.png")
plt.close()

In [23]:
##时间
fig = plt.figure(figsize=(9, 6))
ax = fig.subplots(1, 1)
plt.plot(t, pc[:, 0], label='Ev=' + str(var[0] * 100)[:2] + '%')
ax.set_yticks(np.arange(-2, 2.6, 0.2))
plt.legend()
plt.grid()
ax.set_title('PC1')
plt.savefig("D:\\grads\\TongJi\\ex6\\PC1.png")
plt.close()

In [24]:
fig = plt.figure(figsize=(9, 6))
ax = fig.subplots(1, 1)
plt.plot(t, pc[:, 1], label='Ev=' + str(var[1] * 100)[:2] + '%')
ax.set_yticks(np.arange(-2, 2.6, 0.2))
plt.legend()
ax.set_title('PC2')
plt.savefig("D:\\grads\\TongJi\\ex6\\PC2.png")
plt.close()

In [25]:
yichangtp = tp[13, :, :] - avetp

In [26]:
##合成分析
##1983，1992 1998 为Elnino年
##1985 1989 1999 2000 2008 为Lanina年
tnino = np.array([4, 13, 19])
tnina = np.array([6, 10, 20, 21, 29])
ninos2 = np.zeros((len(lat), len(lon)))
ninas2 = np.zeros((len(lat), len(lon)))
avenino = np.zeros((len(lat), len(lon)))
avenina = np.zeros((len(lat), len(lon)))
for t_i in tnino:
    avenino = avenino + tp[t_i, :, :]
avenino = avenino / len(tnino)
for t_i in tnina:
    avenina = avenina + tp[t_i, :, :]
avenina = avenina / len(tnina)
for t_i in tnino:
    ninos2 = ninos2 + (tp[t_i, :, :] - avenino) * (tp[t_i, :, :] - avenino)
for t_i in tnina:
    ninas2 = ninas2 + (tp[t_i, :, :] - avenina) * (tp[t_i, :, :] - avenina)

In [27]:
tjianyan = (avenino - avenina) / np.sqrt(ninas2 / len(tnina) + ninos2 / len(tnino))

In [28]:
###海温气候场
ax, fig = createmap()
colorbar = ax.contourf(lon, lat, avetp, cmap='bwr', transform=ccrs.PlateCarree())
plt.colorbar(colorbar, extendrect='True', pad=0.1, fraction=0.04, shrink=1, location="bottom")
lines = ax.contour(lon, lat, avetp, colors='k', linestyles='-', transform=ccrs.PlateCarree())
plt.clabel(lines, inline=True, fontsize=8, fmt='%.1f')
ax.set_title('海温气候场')
plt.tight_layout()
plt.savefig("D:\\grads\\TongJi\\ex6\\qihouchang.png")
plt.close()

In [29]:
###1992年El nino 海温异常场
ax, fig = createmap()
colorbar = ax.contourf(lon, lat, yichangtp, cmap='bwr', transform=ccrs.PlateCarree())
plt.colorbar(colorbar, extendrect='True', pad=0.1, fraction=0.04, shrink=1, location="bottom")
lines = ax.contour(lon, lat, yichangtp, colors='k', linestyles='-', transform=ccrs.PlateCarree())
plt.clabel(lines, inline=True, fontsize=8, fmt='%.1f')
ax.set_title('1992年海温异常场')
plt.tight_layout()
plt.savefig("D:\\grads\\TongJi\\ex6\\yichang.png")
plt.close()

In [30]:
### t检验
ax, fig = createmap()
colorbar = ax.contourf(lon, lat, tjianyan, cmap='bwr', transform=ccrs.PlateCarree())
plt.colorbar(colorbar, extendrect='True', pad=0.1, fraction=0.04, shrink=1, location="bottom")
lines = ax.contour(lon, lat, tjianyan, colors='k', linestyles='-', transform=ccrs.PlateCarree())
plt.clabel(lines, inline=True, fontsize=8, fmt='%.1f')
ax.set_title('t检验')
plt.tight_layout()
plt.savefig("D:\\grads\\TongJi\\ex6\\tjianyan.png")
plt.close()

In [31]:
##第二问1992年/1983年
X = np.zeros((len(lat), len(lon)))
for i in range(10):
    X = X + eof[i, :, :] * pc[4 , i]##13-1991

In [32]:
ax, fig = createmap()
colorbar = ax.contourf(lon, lat, X, cmap='bwr', transform=ccrs.PlateCarree())
plt.colorbar(colorbar, extendrect='True', pad=0.1, fraction=0.04, shrink=1, location="bottom")
lines = ax.contour(lon, lat, X, colors='k', linestyles='-', transform=ccrs.PlateCarree())
plt.clabel(lines, inline=True, fontsize=8, fmt='%.1f')
ax.set_title('1983年异常重构结果')
plt.tight_layout()
plt.savefig("D:\\grads\\TongJi\\ex6\\1991yichang_chonggou1.png")
plt.close()

In [33]:
ax, fig = createmap()
colorbar = ax.contourf(lon, lat, tp[4, :, :]-avetp, cmap='bwr', transform=ccrs.PlateCarree())
plt.colorbar(colorbar, extendrect='True', pad=0.1, fraction=0.04, shrink=1, location="bottom")
lines = ax.contour(lon, lat, tp[4, :, :]-avetp, colors='k', linestyles='-', transform=ccrs.PlateCarree())
plt.clabel(lines, inline=True, fontsize=8, fmt='%.1f')
ax.set_title('1983年原始场')
plt.tight_layout()
plt.savefig("D:\\grads\\TongJi\\ex6\\1992yuanshi1.png")
plt.close()

In [34]:
###主分量分析法
iost=iosst['st'][:,:,:]
lat1=iosst['lat'][:]
lon1=iosst['lon'][:]
iost=iost.where(iost<9999,np.NAN)
aveiost=iost.mean(dim='time')
jupingio = np.zeros((len(t), len(lat1), len(lon1)))
for t_i in range(len(t)):
    jupingio[t_i, :, :] = iost[t_i, :, :] - aveiost
a=Eof(jupingio)
iopc=a.pcs(npcs=1,pcscaling=1)
iovar=a.varianceFraction(neigs=1)
fig = plt.figure(figsize=(9, 6))
ax = fig.subplots(1, 1)
plt.plot(t, iopc[:, 0], label='Ev=' + str(iovar[0] * 100)[:2] + '%')
ax.set_yticks(np.arange(-3.6, 1.6, 0.2))
plt.legend()
ax.set_title('iopc1')
plt.savefig("D:\\grads\\TongJi\\ex6\\iopc1.png")
plt.grid()
plt.close()